In [32]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [33]:
@inbounds U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
@inbounds σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
@inbounds g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )

@inbounds function TM(u, p, t)
    
    U_ = U(u[3], p)
    
    du1 = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du2 = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2], p)
    
    return SVector(du1, du2, du3)
end
@inbounds function jacob_TM_(u, p, t)
    
    U(y, p, exp50) = p[8] + p[9] / ( 1.0 + exp50 )
    U_y(y, p, exp50) = (50.0 * p[9] * exp50) / (1.0 + exp50)^2
    g(E, x, y, p, U_) = exp((p[5]  * U_ * x * E + p[11]) / p[1])
    σ_der(x, p) = exp( (-20.0) * (x - p[6]) )
    exp50 = exp(-50.0 * (u[3] - p[7]))
    
    U_ = U(u[3], p, exp50)
    Uy = U_y(u[3], p, exp50)
    g_ = g(u[1], u[2], u[3], p, U_)
    σ_deri = σ_der(u[2], p)
    
    g_plus = 1.0 + g_
    g_mult = g_ * U_
    g_plus_mult = p[2] * (g_plus)
    u1p5 = p[5] * u[1]
    Uyu2 = Uy * u[2]
    
    E_E = (-1.0 + ((J * u[2] * g_mult)) / (g_plus) ) / p[2]
    E_x = (u1p5 * g_mult) / (g_plus_mult)
    E_y = (u1p5 * Uyu2 * g_) / (g_plus_mult)
    
    x_E = -U_ * u[2]
    x_x = -1.0 / p[3] - U_ * u[1]
    x_y = -Uyu2 * u[1]
    
    y_x = 20.0 * p[10] * σ_deri / (1.0 + σ_deri)^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end;

In [34]:
Pxindex = 11; Pyindex = 8
Pxlabel = "I0"; Pylabel = "U0"
Pxrange = range( -0.5, -2.0,  length = 80 )
Pyrange = range( 0.22,  0.55, length = 80 )

0.22:0.004177215189873418:0.55

In [35]:
Λs = zeros(length(Pxrange), length(Pyrange), 3)
u0s = zeros(length(Pxrange), length(Pyrange), 3);

In [58]:
file_name_LS = "LSs_TEST.jld"
file_name_u0s = "u0s_TEST.jld"

"u0s_TEST.jld"

In [59]:
t = 500; tt = 500; tstep = 0.001
time_attract = 100.0

trange = range(0.0, t, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [60]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.22;  const ΔU0 = 0.305; I0 = -0.5
p = [α, τ, τD, τy, J, xthr, ythr, 0, ΔU0, β, 0]

u0 = [8.39353, 0.73803, 0.429055]

In [72]:
const Px_start = Pxrange[1]
const Py_start = Pyrange[1]

function init_ds_Px(p, Px, Pxindex, u0)
    p[Pxindex] = Px; p[Pyindex] = Py_start
    ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
    return ds
end
function init_ds_Py(p, Px, Pyindex, u0)
    p[Pyindex] = Py; p[Pxindex] = Px_start
    ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
    return ds
end
function init_ds(p, Pxindex, Pyindex, Px, Py, u0)
    p[Pyindex] = Py; p[Pxindex] = Px
    ds = CoupledODEs(TM, u0, p,  diffeq = integ_set)
    return ds
end
function goto_attractor(ds, t, tt)
    tr, _ = trajectory(ds, t; Ttr = tt)
    u0 = tr[end]
    return u0
end
function spectrum(ds)
     ΛΛ = lyapunovspectrum(ds, t)
end
function save_output_Px(idx_Px, ΛΛ, u0)
    Λs[idx_Px, 1, :] = ΛΛ
    u0s[idx_Px, 1, :] = u0
end
function save_output_Py(idx_Py, ΛΛ, u0)
    Λs[1, idx_Py, :] = ΛΛ
    u0s[1, idx_Py, :] = u0
end
function save_output(idx_Px, idx_Py, ΛΛ, u0)
    Λs[idx_Px, idx_Py, :] = ΛΛ
    u0s[idx_Px, idx_Py, :] = u0
end
function save_tofile()
    save(file_name_LS, "data", Λs)
    save(file_name_u0s, "data", u0s)
end

function outputPx(idxPx, Px, u0)
    println("Индекс $Pxlabel: $idxPx; $Pxlabel: $Px");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function outputPy(idxPx, Px, u0)
    println("Индекс $Pylabel: $idxPx; $Pylabel: $Px");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_Px, idx_Py, Px, Py, u0)
    println("Индекс $Pxlabel: $idx_Px; $Pxlabel: $Px");flush(stdout)
    println("Индекс $Pylabel: $idx_Py; $Pylabel: $Py");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function outputIter(ΛΛ, u0)
    println("Спектр: $ΛΛ");flush(stdout)
    println("Последняя точка: $u0");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [64]:
# INHERITANCE RIGHT-LEFT

for (idx_Px, Px) in enumerate(Pxrange)
    
    if idx_Px == 1 # Проверка на первую итерацию
        # Если использовать while условие не нужно!
        global u0_lc = u0
    end
    
    outputPx(idx_Px ,Px, u0_lc)
    
    ds = init_ds_Px(p, Px, Pxindex, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract, tt) # Выход на аттрактор
    ds = init_ds_Px(p, Px, Pxindex, u0_lc)
    ΛΛ = spectrum(ds) # Вычисление спектра
    
    outputIter(ΛΛ, u0_lc)
    
    save_output_Px(idx_Px, ΛΛ, u0_lc) # Сохранение значений
    
    separate()
    #save_tofile()
end

Индекс I0: 1; I0: -0.5
Начальные условия: [8.39353, 0.73803, 0.429055]
Спектр: [-3.1456639622497162, -3.1466036455689594, -7.1984027958775565]
Последняя точка: [9.61839650326879, 0.7364220338699033, 0.42819908049470906]

---------
Индекс I0: 2; I0: -0.5189873417721519
Начальные условия: [9.61839650326879, 0.7364220338699033, 0.42819908049470906]
Спектр: [-3.0943569242502016, -3.095024832141016, -7.141648892808943]
Последняя точка: [9.5965642626436, 0.7364974719296596, 0.4285657355017605]

---------
Индекс I0: 3; I0: -0.5379746835443038
Начальные условия: [9.5965642626436, 0.7364974719296596, 0.4285657355017605]
Спектр: [-3.046865820846115, -3.0396967938343606, -7.084108904279352]
Последняя точка: [9.574689123340256, 0.736574048670285, 0.4289380008306229]

---------
Индекс I0: 4; I0: -0.5569620253164557
Начальные условия: [9.574689123340256, 0.736574048670285, 0.4289380008306229]
Спектр: [-2.9914419955330267, -2.992380344139819, -7.025763158467898]
Последняя точка: [9.552769597097967, 0


---------
Индекс I0: 31; I0: -1.0696202531645569
Начальные условия: [8.959572989439883, 0.739289844273244, 0.44218651155777206]
Спектр: [-1.660332609635432, -1.6454891010210286, -5.042883145225747]
Последняя точка: [8.93538186139925, 0.7394276443189062, 0.44286095565224043]

---------
Индекс I0: 32; I0: -1.0886075949367089
Начальные условия: [8.93538186139925, 0.7394276443189062, 0.44286095565224043]
Спектр: [-1.6083215122169234, -1.6054411962289403, -4.9488471375761005]
Последняя точка: [8.911029836869899, 0.7395696860660363, 0.4435563651158114]

---------
Индекс I0: 33; I0: -1.1075949367088607
Начальные условия: [8.911029836869899, 0.7395696860660363, 0.4435563651158114]
Спектр: [-1.5605711237316802, -1.5620047699876758, -4.852769886495691]
Последняя точка: [8.88650618584015, 0.739716251364373, 0.44427413625308343]

---------
Индекс I0: 34; I0: -1.1265822784810127
Начальные условия: [8.88650618584015, 0.739716251364373, 0.44427413625308343]
Спектр: [-1.51635084885224, -1.51596229367

Последняя точка: [8.002508362457194, 0.74950481555375, 0.4925488570256468]

---------
Индекс I0: 61; I0: -1.639240506329114
Начальные условия: [8.002508362457194, 0.74950481555375, 0.4925488570256468]
Спектр: [-0.4550554809654403, -0.45467503736552506, -0.8563466227270958]
Последняя точка: [7.931426878433536, 0.7508743591033815, 0.4993279672706267]

---------
Индекс I0: 62; I0: -1.6582278481012658
Начальные условия: [7.931426878433536, 0.7508743591033815, 0.4993279672706267]
Спектр: [-0.364142793891125, -0.3663969424631755, -0.6766102699800006]
Последняя точка: [7.843969668915398, 0.7526741739342979, 0.5082340064882049]

---------
Индекс I0: 63; I0: -1.6772151898734178
Начальные условия: [7.843969668915398, 0.7526741739342979, 0.5082340064882049]
Спектр: [-0.23248421085805232, -0.23467306428766888, -0.5190136777740104]
Последняя точка: [7.730466874089428, 0.7551586816524628, 0.5205128466011759]

---------
Индекс I0: 64; I0: -1.6962025316455696
Начальные условия: [7.730466874089428, 0.7

In [73]:
for (idx_Px, Px) in enumerate(Pxrange)
    for (idx_Py, Py) in enumerate(Pyrange)
    
        if idx_Py == 1 # Если использовать while условие не нужно!
            continue
        end
        
        u0_lc = u0s[idx_Px, idx_Py- 1, :]
        
        output(idx_Px, idx_Py, Px, Py, u0_lc)
        
        ds = init_ds(p, Pxindex, Pyindex, Px, Py, u0)
        u0_lc = goto_attractor(ds, time_attract, tt) # Выход на аттрактор
        ds = init_ds(p, Pxindex, Pyindex, Px, Py, u0)
        ΛΛ = spectrum(ds) # Вычисление спектра
        
        outputIter(ΛΛ, u0_lc)
        
        save_output(idx_Px, idx_Py, ΛΛ, u0_lc)
        
        separate()
    end
    # save_tofile()
end

Индекс I0: 1; I0: -0.5
Индекс U0: 2; U0: 0.2241772151898734
Начальные условия: [9.61839650326879, 0.7364220338699033, 0.42819908049470906]
Спектр: [-3.011506880462871, -3.0092543593240375, -7.480980979694042]
Последняя точка: [9.631173287223506, 0.7360883336258764, 0.42657808295033145]

---------
Индекс I0: 1; I0: -0.5
Индекс U0: 3; U0: 0.22835443037974684
Начальные условия: [9.631173287223506, 0.7360883336258764, 0.42657808295033145]
Спектр: [-2.8850553394481175, -2.883230450557306, -7.744298044717781]
Последняя точка: [9.643408542144734, 0.735768782587182, 0.42502721776992985]

---------
Индекс I0: 1; I0: -0.5
Индекс U0: 4; U0: 0.23253164556962025
Начальные условия: [9.643408542144734, 0.735768782587182, 0.42502721776992985]
Спектр: [-2.766465501802144, -2.7676940416311635, -7.98885399061851]
Последняя точка: [9.65516818675645, 0.7354616584000895, 0.423537982829473]

---------
Индекс I0: 1; I0: -0.5
Индекс U0: 5; U0: 0.23670886075949368
Начальные условия: [9.65516818675645, 0.7354616

Последняя точка: [9.88606689581905, 0.7294323085093098, 0.3946016672028055]

---------
Индекс I0: 1; I0: -0.5
Индекс U0: 30; U0: 0.3411392405063291
Начальные условия: [9.88606689581905, 0.7294323085093098, 0.3946016672028055]
Спектр: [-2.010657808303784, -2.0093205445984754, -9.717855214816954]
Последняя точка: [9.894832538340001, 0.72920345035556, 0.3935159282513277]

---------
Индекс I0: 1; I0: -0.5
Индекс U0: 31; U0: 0.3453164556962025
Начальные условия: [9.894832538340001, 0.72920345035556, 0.3935159282513277]
Спектр: [-2.0586676081880446, -2.0477097832266353, -9.639468115956186]
Последняя точка: [9.903680882047269, 0.7289724354538277, 0.392420990202246]

---------
Индекс I0: 1; I0: -0.5
Индекс U0: 32; U0: 0.34949367088607597
Начальные условия: [9.903680882047269, 0.7289724354538277, 0.392420990202246]
Спектр: [-2.1021034680782993, -2.1017871885924015, -9.55005969801312]
Последняя точка: [9.912626195495209, 0.7287388913381319, 0.3913151296448235]

---------
Индекс I0: 1; I0: -0.5
И

Индекс U0: 57; U0: 0.4539240506329114
Начальные условия: [10.243196553529208, 0.7201100031443525, 0.35129872268978823]
Спектр: [-2.73612205935663, -5.674944506174096, -5.671849168450279]
Последняя точка: [10.273273953536583, 0.7193250382631992, 0.347748775103256]

---------
Индекс I0: 1; I0: -0.5
Индекс U0: 58; U0: 0.4581012658227848
Начальные условия: [10.273273953536583, 0.7193250382631992, 0.347748775103256]
Спектр: [-2.3065561428165102, -5.905463792429915, -5.902852584101259]
Последняя точка: [10.308205618050673, 0.7184134141493267, 0.3436468001925345]

---------
Индекс I0: 1; I0: -0.5
Индекс U0: 59; U0: 0.4622784810126582
Начальные условия: [10.308205618050673, 0.7184134141493267, 0.3436468001925345]
Спектр: [-1.8865840267707408, -6.13468681662451, -6.131703016510396]
Последняя точка: [10.34984021604719, 0.7173268998271183, 0.338787709302856]

---------
Индекс I0: 1; I0: -0.5
Индекс U0: 60; U0: 0.46645569620253163
Начальные условия: [10.34984021604719, 0.7173268998271183, 0.338787

LoadError: InterruptException: